In [1]:
import numpy as np
from garuda.ops import geo_to_webm_pixel, webm_pixel_to_geo, get_sentinel2_visual
from garuda.od import yolo_aa_to_geo

import rasterio as rio
import numpy as np
from numpy import ndarray
import pyproj
from beartype.typing import Tuple, Union
import warnings
from beartype import beartype
from jaxtyping import Float, jaxtyped, Int
from beartype.typing import List

import planetary_computer as pc
from pystac_client import Client
from pystac.extensions.eo import EOExtension as eo
from shapely.geometry import box
from rioxarray.merge import merge_arrays
import rioxarray

In [4]:
@jaxtyped(typechecker=beartype)
def check(a: List[Float[ndarray, "n m"]]):
    return a

check([np.random.rand(3, 4), np.random.rand(5, 6)])

[array([[0.90476524, 0.10874211, 0.24224782, 0.43911673],
        [0.45333331, 0.56400373, 0.27479027, 0.91849912],
        [0.73771259, 0.66894545, 0.44491045, 0.41501535]]),
 array([[0.52512411, 0.08684256, 0.55091178, 0.61461792, 0.17619967,
         0.18451527],
        [0.29144337, 0.54634595, 0.95109858, 0.38502176, 0.20480036,
         0.84316217],
        [0.9438109 , 0.04602024, 0.4003533 , 0.64806409, 0.27990322,
         0.55300185],
        [0.2204828 , 0.1696394 , 0.39441362, 0.22283286, 0.30059866,
         0.17681949],
        [0.84324772, 0.13621688, 0.33615577, 0.79379001, 0.53483733,
         0.18615876]])]

In [2]:
import torch
boxes = torch.rand(9, 5)
print(f"boxes: {boxes.shape}")
gbbs = torch.cat((boxes[:, 2:4].pow(2) / 12, boxes[:, 4:]), dim=-1)
print(f"gbbs: {gbbs.shape}")
a, b, c = gbbs.split(1, dim=-1)
cos = c.cos()
sin = c.sin()
cos2 = cos.pow(2)
sin2 = sin.pow(2)
ans = a * cos2 + b * sin2, a * sin2 + b * cos2, (a - b) * cos * sin
print(ans[0].mean(), ans[1].mean(), ans[2].mean())

boxes = boxes.numpy()
print(f"boxes: {boxes.shape}")
gbbs = np.concatenate((boxes[:, 2:4] ** 2 / 12, boxes[:, 4:]), axis=-1)
print(f"gbbs: {gbbs.shape}")
a, b, c = np.split(gbbs, [1, 2], axis=-1)
cos = np.cos(c)
sin = np.sin(c)
cos2 = cos ** 2
sin2 = sin ** 2
ans = a * cos2 + b * sin2, a * sin2 + b * cos2, (a - b) * cos * sin
print(ans[0].mean(), ans[1].mean(), ans[2].mean())

boxes: torch.Size([9, 5])
gbbs: torch.Size([9, 3])
tensor(0.0357) tensor(0.0295) tensor(0.0031)
boxes: (9, 5)
gbbs: (9, 3)
0.035692394 0.02945243 0.0031444672


In [21]:
from ultralytics.utils.metrics import probiou
from ultralytics.utils.ops import xyxyxyxy2xywhr
from garuda.ops import obb_iou
from tqdm.notebook import tqdm

for i in tqdm(range(10000)):
    obb1 = torch.rand(13, 9)
    obb2 = torch.rand(13, 9)
    xywhr1 = xyxyxyxy2xywhr(obb1[:, 1:9])
    xywhr2 = xyxyxyxy2xywhr(obb2[:, 1:9])

    iou = probiou(xywhr1, xywhr2)
    garuda_iou = obb_iou(obb1.numpy(), obb2.numpy())
    assert np.allclose(iou.ravel().numpy(), garuda_iou, atol=1e-3)

  0%|          | 0/10000 [00:00<?, ?it/s]

In [2]:
lat_c = 33.7756
lon_c = -84.3963
time_of_interest = "2023-01-01/2023-01-31"
max_cloud_cover = 10.0
max_items = 10

# polygon = box(lon_c - 0.01, lat_c - 0.01, lon_c + 0.01, lat_c + 0.01)
    
# catalog = Client.open("https://planetarycomputer.microsoft.com/api/stac/v1", modifier=pc.sign_inplace)

# search = catalog.search(
#     collections=["sentinel-2-l2a"],
#     intersects=polygon,
#     datetime=time_of_interest,
#     query={"eo:cloud_cover": {"lt": max_cloud_cover}},
#     max_items=max_items,
# )

# items = search.item_collection()
# least_cloud_cover_item = min(items, key=lambda x: eo.ext(x).cloud_cover)
# href = least_cloud_cover_item.assets["visual"].href
# visual_raster = rioxarray.open_rasterio(pc.sign(href))
# visual_raster
ds = get_sentinel2_visual(lat_c, lon_c, 128, 128, time_of_interest, max_cloud_cover)
ds

<xarray.DataArray (band: 3, y: 128, x: 128)> Size: 49kB
[49152 values with dtype=uint8]
Coordinates:
  * band         (band) int64 24B 1 2 3
  * x            (x) float64 1kB 7.405e+05 7.405e+05 ... 7.417e+05 7.417e+05
  * y            (y) float64 1kB 3.741e+06 3.741e+06 ... 3.74e+06 3.74e+06
    spatial_ref  int64 8B 0
Attributes: (12/30)
    AREA_OR_POINT:       Area
    _FillValue:          0
    scale_factor:        1.0
    add_offset:          0.0
    timestamp:           2023-01-24 16:25:51.024000+00:00
    visual_href:         https://sentinel2l2a01.blob.core.windows.net/sentine...
    ...                  ...
    granule-metadata:    https://sentinel2l2a01.blob.core.windows.net/sentine...
    inspire-metadata:    https://sentinel2l2a01.blob.core.windows.net/sentine...
    product-metadata:    https://sentinel2l2a01.blob.core.windows.net/sentine...
    datastrip-metadata:  https://sentinel2l2a01.blob.core.windows.net/sentine...
    tilejson:            https://planetarycomputer.microsoft.com/api/data/v1/...
    rendered_preview:    https://planetarycomputer.microsoft.com/api/data/v1/...

In [4]:
ds.rio.crs

CRS.from_epsg(32616)

In [5]:
ds.rio.to_raster("test.tif")

In [6]:
loaded_tif = rio.open("test.tif")
# get crs
print(loaded_tif.crs)
# get timestamp
print(loaded_tif.tags())

EPSG:32616
{'AOT': 'https://sentinel2l2a01.blob.core.windows.net/sentinel2-l2/16/S/GC/2023/01/24/S2A_MSIL2A_20230124T162551_N0400_R040_T16SGC_20230126T150708.SAFE/GRANULE/L2A_T16SGC_A039648_20230124T162913/IMG_DATA/R10m/T16SGC_20230124T162551_AOT_10m.tif?st=2024-08-24T10%3A36%3A57Z&se=2024-08-25T11%3A21%3A57Z&sp=rl&sv=2024-05-04&sr=c&skoid=9c8ff44a-6a2c-4dfb-b298-1c9212f64d9a&sktid=72f988bf-86f1-41af-91ab-2d7cd011db47&skt=2024-08-25T02%3A23%3A12Z&ske=2024-09-01T02%3A23%3A12Z&sks=b&skv=2024-05-04&sig=auUiWOkdO/NOaTlBNCWm/5KMLyYFd2phAcVDusZCHRI%3D', 'AREA_OR_POINT': 'Area', 'B01': 'https://sentinel2l2a01.blob.core.windows.net/sentinel2-l2/16/S/GC/2023/01/24/S2A_MSIL2A_20230124T162551_N0400_R040_T16SGC_20230126T150708.SAFE/GRANULE/L2A_T16SGC_A039648_20230124T162913/IMG_DATA/R60m/T16SGC_20230124T162551_B01_60m.tif?st=2024-08-24T10%3A36%3A57Z&se=2024-08-25T11%3A21%3A57Z&sp=rl&sv=2024-05-04&sr=c&skoid=9c8ff44a-6a2c-4dfb-b298-1c9212f64d9a&sktid=72f988bf-86f1-41af-91ab-2d7cd011db47&skt=2024-08